In [1]:
# import library
import json
from copy import deepcopy

import numpy as np
import marmote.core as mc
import marmote.markovchain as mmc
import networkx as nx

In [2]:
# read jani file
jani_file = "crowds.jani"

with open(jani_file, 'r', encoding='utf-8') as f:
    jani_data = json.load(f)

In [3]:
# list of variables
variables = jani_data["variables"]
dim = len(variables)
varDict = {}
for k, var in enumerate(variables):
    varDict[var['name']] = k

print(varDict)

{'runCount': 0, 'lastSeen': 1, 'observe0': 2, 'observe1': 3, 'observe2': 4, 'observe3': 5, 'observe4': 6, 'observe5': 7, 'observe6': 8, 'observe7': 9, 'observe8': 10, 'observe9': 11, 'observe10': 12, 'observe11': 13, 'observe12': 14, 'observe13': 15, 'observe14': 16, 'observe15': 17, 'observe16': 18, 'observe17': 19, 'observe18': 20, 'observe19': 21, 'launch': 22, 'new': 23, 'start': 24, 'run': 25, 'good': 26, 'bad': 27, 'recordLast': 28, 'badObserve': 29, 'deliver': 30, 'done': 31}


In [29]:
class Data:
    def __init__(self, jani_data):
        self.jani_data = jani_data
        self.variables = jani_data["variables"]
        self.varDict = {}
        for k, var in enumerate(self.variables):
            self.varDict[var['name']] = k
        self.constants = jani_data["constants"]
        self.constDict = {}
        for k, const in enumerate(self.constants):
            self.constDict[const['name']] = k
        
        #asseignment for two constants
        self.constants[2]["value"] = 3
        self.constants[3]["value"] = 5
            
    def getVariableType(self, variable):
        idx = self.varDict[variable]
        type = variables[idx]['type']
        if type == 'bool':
            return 'bool'
        else:
            return type["base"]
        
    def getVariableIdx(self, variable):
        return self.varDict[variable]
    
    def getConstantValue(self, const):
        idx = self.constDict[const]
        return self.constants[idx]['value']
        
data = Data(jani_data)
data.getVariableType("launch")
data.getConstantValue("badC")
data.getConstantValue("TotalRuns")

3

In [32]:
def satisfyExpression(exp, node):
    if isinstance(exp, str):
        if data.getVariableType(exp) == 'bool':
            idx = data.getVariableIdx(exp)
            return node[idx]
    if exp['op'] == '∧':
        return satisfyExpression(exp['left'], node) and satisfyExpression(exp['right'], node)
    idx = data.getVariableIdx(exp['left'])
    if exp['op'] == '=':
        return calValue(node[idx]) == calValue(exp['right'])
    elif exp['op'] == '<':
        return calValue(node[idx]) < calValue(exp['right'])
    elif exp['op'] == '>':
        return calValue(node[idx]) > calValue(exp['right'])

In [19]:
root = tuple(var['initial-value'] for var in variables)

In [ ]:
constants = jani_data["constants"]

In [30]:
def calValue(value):
    if isinstance(value, int) or isinstance(value, float) or isinstance(value, bool):
        return value
    if isinstance(value, str):
        return data.getConstantValue(value)
    else:
        if value["op"] == "+":
            
        elif value["op"] == "-":
            

In [33]:
openNode = [root]
closedNode = []

G = nx.DiGraph()
automata = jani_data["automata"]
edges = automata[0]["edges"]
while openNode:
    x = openNode.pop(0)
    for edge in edges:
        guard = edge['guard']['exp']
        if satisfyExpression(guard, x):
            closedNode.append(x)
            nbdest = 0
            for dest in edge["destinations"]:
                nbdest += 1
                print('nbdest : ',nbdest)
                newNode = deepcopy(x)
                for assignment in dest["assignments"]:
                    idx = varDict[assignment['ref']]
                    print('x: ', x)
                    newNode = newNode[:idx]+(assignment['value'],)+newNode[idx+1:]
                print('newNode: ', newNode)
                if not newNode in openNode and not newNode in closedNode:
                    openNode.append(newNode)
                try:
                    val = dest["probability"]["exp"]
                except KeyError:
                    val = 1
                print('val: ', val)
                G.add_edge(x, newNode, weight=val)

            break
        else:
            continue

print(closedNode)

nbdest :  1
x:  ('TotalRuns', 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, True, False, False, False, False, False, False, False, False, False)
x:  ('TotalRuns', 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, True, False, False, False, False, False, False, False, False, False)
x:  ('TotalRuns', 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, True, False, False, False, False, False, False, False, False, False)
newNode:  ('TotalRuns', 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, False, True, False, False, False, False, False, False, False, False)
val:  1
nbdest :  1
x:  ('TotalRuns', 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, False, True, False, False, False, False, False, False, False, False)
x:  ('TotalRuns', 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, False, True, False, False, False, False, False, False, False, False)
x:  ('TotalRuns', 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

TypeError: unhashable type: 'dict'